# Object Tracking using Recurrent YOLO

In [ ]:
# import the required modules
import os
import sys
import cv2
import ROLO_utils as utils

## Load Test Data Sequence

In [ ]:
# test sequence parameters
num_steps = 6
w_img, h_img = [360, 240]
sequence_name = 'Car4'

# test sequence paths
img_fold_path = os.path.join('data', sequence_name, 'img/')
gt_file_path = os.path.join('data', sequence_name, 'groundtruth_rect.txt')
yolo_out_path = os.path.join('data', sequence_name, 'yolo_out/')
rolo_out_path = os.path.join('data', sequence_name, 'rolo_out_test/')

# load test sequence data
paths_imgs = utils.load_folder(img_fold_path)
lines = utils.load_dataset_gt(gt_file_path)
paths_rolo = utils.load_folder(rolo_out_path)

## Drawing Function

In [ ]:
# function to draw ground truth, YOLO detection, ROLO tracking output
def draw_3_locations(img, gt_location, yolo_location, rolo_location):
    img_cp = img.copy()
    for i in range(3):  # b-g-r channels
        if i == 0:
            location = gt_location
            color = (0, 0, 255)   # red for gt
        elif i == 1:
            location = yolo_location
            color = (255, 0, 0)   # blue for yolo
        elif i == 2:
            location = rolo_location
            color = (0, 255, 0)   # green for rolo
        x = int(location[0])
        y = int(location[1])
        w = int(location[2])
        h = int(location[3])
        if i == 0:
            #cv2.rectangle(img_cp, (x, y), (x+w, y+h), color, 2)
            pass
        elif i == 1:
            cv2.rectangle(img_cp, (x-w//2, y-h//2), (x+w//2, y+h//2), color, 2)
            pass
        elif i == 2:
            cv2.rectangle(img_cp, (x-w//2, y-h//2), (x+w//2, y+h//2), color, 2)
            pass

    return img_cp

## Evaluate Test Sequence

In [ ]:
total = 0
rolo_avgloss = 0
yolo_avgloss = 0
for i in range(len(paths_rolo) - num_steps):
    id = i + 1
    test_id = id + num_steps - 2  # * num_steps + 1

    path = paths_imgs[test_id]
    img = utils.file_to_img(path)
        
    if(img is None):
        break

    gt_location = utils.find_gt_location(lines, test_id - 1)
    print('gt: ' + str(test_id), gt_location)

    yolo_location = utils.find_yolo_location(yolo_out_path, test_id)
    yolo_location = utils.locations_normal(w_img, h_img, yolo_location)
    print('yolo: ', yolo_location)

    rolo_location = utils.find_rolo_location(rolo_out_path, test_id)
    rolo_location = utils.locations_normal(w_img, h_img, rolo_location)
    print('rolo: ', rolo_location)

    frame = draw_3_locations(img, gt_location, yolo_location, rolo_location)

    cv2.imshow('frame', frame)
    cv2.waitKey(50)

    rolo_loss = utils.cal_rolo_IOU(rolo_location, gt_location)
    rolo_avgloss += rolo_loss
    yolo_loss = utils.cal_yolo_IOU(yolo_location, gt_location)
    yolo_avgloss += yolo_loss
    total += 1

In [ ]:
rolo_avgloss /= total
yolo_avgloss /= total
print("yolo_avg_iou = ", yolo_avgloss)
print("rolo_avg_iou = ", rolo_avgloss)
cv2.destroyAllWindows()